In [1]:
from datasets import load_dataset, Dataset, concatenate_datasets

/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# DuReader2.0

In [ ]:
input_path = ""

In [ ]:
dataset = load_dataset("json", data_files=input_path)["train"]

In [ ]:
total, baike, zhidao, tieba, zhihu = 0, 0, 0, 0, 0
for data in dataset:
    for document in data["documents"]:
        total += 1
        title = document["title"]
        if title.endswith("_百度百科"):
            baike += 1
        elif title.endswith("_百度知道"):
            zhidao += 1
        elif title.endswith("_百度贴吧"):
            tieba += 1
        elif title.endswith(" - 知乎"):
            zhihu += 1
total, baike, zhidao, tieba, zhihu

## Baidu Tieba

In [2]:
input_path = ""
output_path = ""

In [3]:
dataset = load_dataset("json", data_files=input_path)["train"]

Generating train split: 136208 examples [00:23, 5911.90 examples/s]


In [4]:
dataset_df = dataset.to_pandas()

In [5]:
processed_data = []

In [6]:
for _, row in dataset_df.iterrows():
    documents = row["documents"]
    for document in documents:
        if document["title"].endswith("_百度贴吧"):
            title = document["title"].removesuffix("_百度贴吧")
            if '_' in title:
                title = title.rsplit('_', 1)[0]
            elif '【' in title:
                title = title.rsplit('【', 1)[0]
            else:
                continue
            if len(document["paragraphs"]) >= 2:
                text = document["paragraphs"][1]
                if text == "下载贴吧APP看高清直播、视频!" or text == "登录百度帐号" or text == "我的游戏 推荐游戏" \
                        or "活动截止" in text or text == "贴吧页面意见反馈 违规贴吧举报反馈通道 贴吧违规信息处理公示" \
                        or text == "百度小说人气榜" or text == "百度移动游戏玩家均可认证(限百度账号),去领取":
                    continue
                processed_data.append({
                    "title": title,
                    "text": document["paragraphs"][1]
                })

In [7]:
dataset = Dataset.from_list(processed_data)

In [8]:
len(dataset)

12949

In [ ]:
dataset.to_parquet(output_path)

## Baidu Zhidao

In [ ]:
input_path = ""
output_path = ""

In [ ]:
dataset = load_dataset("json", data_files=input_path)["train"]

In [ ]:
len(dataset)

In [ ]:
zhidao = 0
for data in dataset:
    documents = data["documents"]
    zhidao += len(documents)
zhidao

In [ ]:
dataset_df = dataset.to_pandas()

In [ ]:
for _, row in dataset_df.iterrows():
    document = row["documents"][0]
    title = document["title"]
    text = document["paragraphs"][0]
    processed_data.append({
        "title": title,
        "text": text
    })

In [ ]:
dataset = Dataset.from_list(processed_data)

In [ ]:
len(dataset)

In [ ]:
dataset.to_parquet(output_path)

# Baidu Baike

In [ ]:
input_path = ""
output_path = ""

In [ ]:
dataset = load_dataset("json", data_files=input_path)["train"]

In [ ]:
len(dataset)

In [ ]:
def process(examples: dict[str, list]) -> dict[str, list]:
    summaries = examples["summary"]
    sections_all = examples["sections"]
    texts = []
    for summary, sections in zip(summaries, sections_all):
        text = ""
        if summary is not None:
            text += summary + "\n"
        for section in sections:
            text += section["title"] + "\n" + section["content"] + "\n"
        text = text.strip("\n")
        texts.append(text)
    examples["text"] = texts
    return examples

In [ ]:
current_column_names = dataset.column_names
current_column_names.remove("title")

In [ ]:
dataset = dataset.map(process, batched=True, batch_size=None, remove_columns=current_column_names)

In [ ]:
dataset.to_parquet(output_path)

# Zhihu

In [1]:
input_dir = ""
output_path = ""

In [3]:
dataset = load_dataset("json", data_dir=input_dir)

In [4]:
test_dataset = dataset["test"]
val_dataset = dataset["validation"]
test_keys = dataset["test"]["uniqueKey"]
val_keys = dataset["validation"]["uniqueKey"]

In [5]:
intersection = set(test_keys) & set(val_keys)

In [6]:
val_dataset = val_dataset.filter(
    lambda examples: [key not in intersection for key in examples["uniqueKey"]],
    batched=True,
    batch_size=None
)

In [7]:
dataset = concatenate_datasets([test_dataset, val_dataset])

In [8]:
def process(examples: dict[str, list]) -> dict[str, list]:
    examples["text"] = [content.removeprefix(title).strip() for title, content in zip(examples["title"], examples["content"])]
    return examples

In [9]:
current_column_names = dataset.column_names
current_column_names.remove("title")

In [10]:
dataset = dataset.map(process, batched=True, batch_size=None, remove_columns=current_column_names)

Map: 100%|██████████| 19991/19991 [00:01<00:00, 14457.05 examples/s]


In [11]:
dataset.to_parquet(output_path)

Creating parquet from Arrow format: 100%|██████████| 20/20 [00:01<00:00, 14.64ba/s]


104411033

# Weibo

In [9]:
input_dir = ""
output_path = ""

In [10]:
dataset = load_dataset("csv", data_dir=input_dir)["train"]

Generating train split: 87800 examples [00:04, 19647.85 examples/s]


In [12]:
from ast import literal_eval


def filter_function(examples: dict[str, list]) -> list[bool]:
    flags = []
    titles = examples["话题"]
    contents_all = examples["内容"]
    unique_titles = set()
    for title, contents in zip(titles, contents_all):
        if len(title) < 4:
            flags.append(False)
            continue
        flag = False
        if contents is not None:
            contents = literal_eval(contents)
            for content in contents:
                comments = content["comments"]
                if len(comments) != 0:
                    for comment in comments:
                        if len(comment) > 10:
                            flag = True
                            break
                if flag:
                    break
        if flag:
            if title in unique_titles:
                flag = False
            unique_titles.add(title)
        flags.append(flag)
    return flags

In [13]:
dataset = dataset.filter(
    filter_function,
    batched=True,
    batch_size=None
)

Filter: 100%|██████████| 87800/87800 [00:09<00:00, 9230.48 examples/s]


In [14]:
len(dataset)

6041

In [21]:
dataset_df = dataset.to_pandas()

In [22]:
processed_data = []

In [31]:
contents = literal_eval(dataset_df.iloc[0]["内容"])
contents[0]["comments"]

['：就怕特没谱随时变卦 毕竟他不太正常',
 '：军工立大功了',
 '：就是只保留跟其他国家一样的10%。其他的再谈。',
 '：90天内暂停实施，90天后还需要谈判，是这个意思吧',
 '：美国佬，尤其是特朗普那个傻蛋，根本不可信！要当心其出尔反尔！',
 '：先辈未曾低头，吾辈岂能弯腰',
 '：谈判和斗争继续。但是给了大家踹口气。存在特没谱随时变卦，所以对美出口企业，一定要抛弃把美国市场作为主要市场的行商策略，当然每个老板有自己的策略，无可厚非。只是要未雨绸缪',
 '：这个确实超预期，只剩下10%的关税了。',
 '：目测二季度订单要爆，延续到三季度头～鬼知道90天之后又有什么幺蛾子，估计四季度会落一点']

In [35]:
for _, row in dataset_df.iterrows():
    title = row["话题"]
    contents = literal_eval(row["内容"])
    for content in contents:
        comments = content["comments"]
        if len(comments) != 0:
            for comment in comments:
                if len(comment) > 10:
                    processed_data.append({
                        "title": title,
                        "text": comment.removeprefix("：")
                    })
dataset = Dataset.from_list(processed_data)

In [36]:
len(dataset)

92399

In [45]:
dataset.to_parquet(output_path)

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 15.70ba/s]


10271029

# RedNote

In [ ]:
input_dir = ""
output_path = ""

In [ ]:
dataset = load_dataset("parquet", data_dir=input_dir)

In [ ]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]
dataset = concatenate_datasets([train_dataset, test_dataset, val_dataset])

In [ ]:
def filter_function(examples: dict[str, list]) -> list[bool]:
    flags = []
    titles = examples["title"]
    unique_titles = set()
    for title in titles:
        if len(title) <= 5 or title in unique_titles:
            flags.append(False)
            continue
        unique_titles.add(title)
        flags.append(True)
    return flags

In [ ]:
dataset = dataset.filter(
    filter_function,
    batched=True,
    batch_size=None
)

In [ ]:
len(dataset)

In [ ]:
new_data = []
for item in dataset:
    title = item['title']
    comments = item['comments']
    if comments:
        for comment in comments:
            if len(comment) >= 10:
                new_data.append({'title': title, 'text': comment})
dataset = Dataset.from_list(new_data)

In [ ]:
len(dataset)

In [ ]:
dataset.to_parquet(output_path)

# Douban

In [ ]:
input_path = ""
output_path = ""

In [ ]:
dataset = load_dataset("csv", data_files=input_path)["train"]

In [ ]:
dataset = dataset.filter(
    lambda examples: [comment and len(comment) >= 10 for comment in examples["Comment"]],
    batched=True,
    batch_size=None
)

In [ ]:
len(dataset)

In [ ]:
dataset.to_parquet(output_path)

# CSL

In [ ]:
input_path = ""
output_path = ""

In [ ]:
import csv
from datasets import Features, Value

csl_features = Features({
    "title": Value("string"),
    "abstract": Value("string"),
    "keywords": Value("string"),
    "category": Value("string"),
    "discipline": Value("string")
})

dataset = load_dataset("csv", data_files=input_path, delimiter="\t", column_names=csl_features.keys(),
                       quoting=csv.QUOTE_NONE)["train"]

In [ ]:
dataset = dataset.remove_columns(["keywords", "category", "discipline"])

In [ ]:
len(dataset)

In [ ]:
dataset.to_parquet(output_path)

# CLTS

In [132]:
train_source_input_path = ""
train_target_input_path = ""
valid_source_input_path = ""
valid_target_input_path = ""
test_source_input_path = ""
test_target_input_path = ""
output_path = ""

In [147]:
train_source_dataset = load_dataset("text", data_files=train_source_input_path)["train"]
train_target_dataset = load_dataset("text", data_files=train_target_input_path)["train"]
valid_source_dataset = load_dataset("text", data_files=valid_source_input_path)["train"]
valid_target_dataset = load_dataset("text", data_files=valid_target_input_path)["train"]
test_source_dataset = load_dataset("text", data_files=test_source_input_path)["train"]
test_target_dataset = load_dataset("text", data_files=test_target_input_path)["train"]

In [157]:
# 合并所有 source 和 target 数据集
source_dataset = concatenate_datasets([train_source_dataset, valid_source_dataset, test_source_dataset])
target_dataset = concatenate_datasets([train_target_dataset, valid_target_dataset, test_target_dataset])

In [158]:
len(source_dataset)

185397

In [159]:
special_white_space_chars = [
    '\t', '\v', '\f', '\xa0', '\u00a0', '\u2000', '\u2001', '\u2002', '\u2003', '\u2004', '\u2005', '\u2006', '\u2007',
    '\u2008', '\u2009', '\u200a', '\u200b', '\u202f', '\u205f', '\u3000', '\ufeff', chr(4447), chr(4448), chr(12644),
    "&nbsp;", "&ensp;", "&rdquo;", "&thinsp;", "&zwnj;", "&zwj;"
]

In [160]:
import re
import string


def process(examples: dict[str, list], is_source_dataset: bool = False) -> dict[str, list]:
    texts = []

    suf_pre = ["（原题", "(原题", "（原文题", "(原文题", "（原标题", "(原标题", "（本文原题", "(本文原题", "（本文标题",
               "(本文标题", "（原文标题", "(原文标题", "原标题：", "原题为"]
    suf_pattern = re.compile(r'(' + '|'.join(re.escape(p) for p in suf_pre) + r').*$')

    for text in examples["text"]:
        # 1. 删除特殊空白字符与首尾空白符
        for char in special_white_space_chars:
            text = text.replace(char, '').strip()

        # 2. 删除中文标点符号周围的空格
        chinese_punctuations = r'[\u3001\u3002\uff0c\uff0e\uff1a\uff1b\uff1f\uff01\u201c\u201d\u2018\u2019\uff08\uff09\u300a\u300b\u3008\u3009\u3010\u3011\u300e\u300f\u300c\u300d\ufe43\ufe44\u3014\u3015]'
        text = re.sub(r'\s+(' + chinese_punctuations + ')', r'\1', text)
        text = re.sub(r'(' + chinese_punctuations + r')\s+', r'\1', text)

        # 3. 删除英文标点符号前的所有连续空格
        english_punctuations = re.escape(string.punctuation)
        text = re.sub(r'\s+([' + english_punctuations + '])', r'\1', text)

        # 4. 如果一个空格不在两个英文字母之间，并且周围也没有另一个空格，删除该空格
        text = re.sub(r'(?<=[^a-zA-Z ]) | (?=[^a-zA-Z ])', '', text)

        # 5. 将剩余的连续空格合并为一个空格
        text = re.sub(r' +', ' ', text)

        # 6. 其他清洗
        if is_source_dataset:
            text = text.removesuffix("(本文来自澎湃新闻，更多原创资讯请下载“澎湃新闻”APP)")
            # 删除以 suf_pre 开头的后缀
            text = suf_pattern.sub('', text).strip()
            # 如 （应被访者要求，文中何莹莹、张春梅、陈婉均为化名，原题为《警惕网拍模特招聘中的高价“模卡”骗局》） 、
            # 澎湃新闻（www.thepaper.cn）记者从上海一中院获悉，该院将择日对此案作出宣判、
            # 、（澎湃新闻记者 俞凯 整理） 等的此类没有被删除

        texts.append(text)

    examples["text"] = texts

    return examples

In [161]:
select_ids = []


def clean_filter(examples: dict[str, list]) -> list[bool]:
    flags = []
    global select_ids
    for i, text in enumerate(examples["text"]):
        if text.startswith("本文原标题："):
            flags.append(False)
        else:
            flags.append(True)
            select_ids.append(i)
    return flags

In [163]:
target_dataset = target_dataset.map(process, batched=True, batch_size=None)
target_dataset = target_dataset.filter(clean_filter, batched=True, batch_size=None)
source_dataset = source_dataset.select(select_ids)
source_dataset = source_dataset.map(process, batched=True, batch_size=None, fn_kwargs={"is_source_dataset": True})

Filter: 100%|██████████| 185397/185397 [00:00<00:00, 333011.31 examples/s]


In [164]:
new_data = []
for i in range(len(target_dataset)):
    new_data.append({
        "title": target_dataset[i]["text"],
        "text": source_dataset[i]["text"]
    })

dataset = Dataset.from_list(new_data)

In [239]:
dataset = dataset.filter(
    lambda examples: [len(text) >= 50 and len(title) >= 10 for title, text in zip(examples["title"], examples["text"])],
    batched=True,
    batch_size=None
)

Filter: 100%|██████████| 185173/185173 [00:02<00:00, 76797.91 examples/s]


In [240]:
len(dataset)

184556

In [241]:
dataset.to_parquet(output_path)

Creating parquet from Arrow format: 100%|██████████| 185/185 [00:14<00:00, 13.02ba/s]


870207223

# Dianping

In [ ]:
input_path = ""
output_path = ""

In [ ]:
dataset = load_dataset("parquet", data_files=input_path)["train"]

In [ ]:
import ast

dataset = dataset.filter(
    lambda examples: [len(ast.literal_eval(object)) != 0 for object in examples["object"]],
    batched=True,
    batch_size=None
)

In [ ]:
len(dataset)

In [ ]:
def process(examples: dict[str, list]) -> dict[str, list]:
    examples["text"] = [
        ast.literal_eval(f'"{sentence.replace('"', '\\"').replace("'", "\\'")}"')
        for sentence in examples["sentence"]
    ]  # e.g. '\\n' -> '\n'
    objects = examples["object"]
    titles = []
    for object_str in objects:
        object_list = ast.literal_eval(object_str)
        titles.append("，".join(object_list))
    examples["title"] = titles
    return examples

In [ ]:
dataset = dataset.map(process, batched=True, batch_size=None, remove_columns=dataset.column_names)

In [ ]:
dataset.to_parquet(output_path)